Test below indicate the model does work on an image with cars

In [1]:
!pip install opencv-python

In [2]:
import tensorflow as tf
import numpy as np
import cv2
import time
from tensorflow import keras
def non_max_suppression(inputs, model_size, max_output_size,
                        max_output_size_per_class, iou_threshold,
                        confidence_threshold):
    bbox, confs, class_probs = tf.split(inputs, [4, 1, -1], axis=-1)
    bbox=bbox/model_size[0]
    scores = confs * class_probs
    boxes, scores, classes, valid_detections = \
        tf.image.combined_non_max_suppression(
        boxes=tf.reshape(bbox, (tf.shape(bbox)[0], -1, 1, 4)),
        scores=tf.reshape(scores, (tf.shape(scores)[0], -1,
                                   tf.shape(scores)[-1])),
        max_output_size_per_class=max_output_size_per_class,
        max_total_size=max_output_size,
        iou_threshold=iou_threshold,
        score_threshold=confidence_threshold
    )
    return boxes, scores, classes, valid_detections
def resize_image(inputs, modelsize):
    inputs= tf.image.resize(inputs, modelsize)
    return inputs
def load_class_names(file_name):
    with open(file_name, 'r') as f:
        class_names = f.read().splitlines()
    return class_names
def output_boxes(inputs,model_size, max_output_size, max_output_size_per_class,
                 iou_threshold, confidence_threshold):
    center_x, center_y, width, height, confidence, classes = \
        tf.split(inputs, [1, 1, 1, 1, 1, -1], axis=-1)
    top_left_x = center_x - width / 2.0
    top_left_y = center_y - height / 2.0
    bottom_right_x = center_x + width / 2.0
    bottom_right_y = center_y + height / 2.0
    inputs = tf.concat([top_left_x, top_left_y, bottom_right_x,
                        bottom_right_y, confidence, classes], axis=-1)
    boxes_dicts = non_max_suppression(inputs, model_size, max_output_size,
                                      max_output_size_per_class, iou_threshold, confidence_threshold)
    return boxes_dicts
def draw_outputs(img, boxes, objectness, classes, nums, class_names):
    boxes, objectness, classes, nums = boxes[0], objectness[0], classes[0], nums[0]
    boxes=np.array(boxes)
    for i in range(nums):
        x1y1 = tuple((boxes[i,0:2] * [img.shape[1],img.shape[0]]).astype(np.int32))
        x2y2 = tuple((boxes[i,2:4] * [img.shape[1],img.shape[0]]).astype(np.int32))
        img = cv2.rectangle(img, (x1y1), (x2y2), (255,0,0), 2)
        img = cv2.putText(img, '{} {:.4f}'.format(
            class_names[int(classes[i])], objectness[i]),
                          (x1y1), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
    return img

In [9]:
model_location='/home/craig/Documents/YOLO-3-OpenCV/yolo3_tf/yolov3_model'
yolo3_model=keras.models.load_model(model_location,compile=False)

In [10]:
model_size = (416, 416,3)
num_classes = 80
class_name = '/home/craig/Documents/YOLO-3-OpenCV/yolo3_tf/coco.names'
max_output_size = 40
max_output_size_per_class= 20
iou_threshold = 0.5
confidence_threshold = 0.5
img_path = "/home/craig/Documents/YOLO-3-OpenCV/yolo3_tf/car.jpeg"

In [ ]:
def main():
    model = yolo3_model
    class_names = load_class_names(class_name)
    image = cv2.imread(img_path)
    image = np.array(image)
    image = tf.expand_dims(image, 0)
    resized_frame = resize_image(image, (model_size[0],model_size[1]))
    pred = model.predict(resized_frame)
    boxes, scores, classes, nums = output_boxes( \
        pred, model_size,
        max_output_size=max_output_size,
        max_output_size_per_class=max_output_size_per_class,
        iou_threshold=iou_threshold,
        confidqence_threshold=confidence_threshold)
    image = np.squeeze(image)
    img = draw_outputs(image, boxes, scores, classes, nums, class_names)
    win_name = 'Image detection'
    cv2.imshow(win_name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    #If you want to save the result, uncommnent the line below:
    #cv2.imwrite('test.jpg', img)
main()

2022-05-12 16:51:37.967078: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-05-12 16:51:37.984152: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2793530000 Hz
QObject::moveToThread: Current thread (0x55831cbd4520) is not the object's thread (0x55831a8e3d30).
Cannot move to target thread (0x55831cbd4520)

QObject::moveToThread: Current thread (0x55831cbd4520) is not the object's thread (0x55831a8e3d30).
Cannot move to target thread (0x55831cbd4520)

QObject::moveToThread: Current thread (0x55831cbd4520) is not the object's thread (0x55831a8e3d30).
Cannot move to target thread (0x55831cbd4520)

QObject::moveToThread: Current thread (0x55831cbd4520) is not the object's thread (0x55831a8e3d30).
Cannot move to target thread (0x55831cbd4520)

QObject::moveToThread: Current thread (0x55831cbd4520) is not the object's thread (0x55831a8e3d30).
Cannot move to target thread (0x55831cbd

In [3]:
img_path = "/home/craig/Documents/YOLO-3-OpenCV/yolo3_tf/test.jpg"
image = cv2.imread(img_path)
image = np.array(image)


In [4]:
print(image)

[[[ 49  79  74]
  [ 58  88  83]
  [ 34  64  59]
  ...
  [  7  16   6]
  [ 11  20  10]
  [ 12  21  11]]

 [[ 49  79  74]
  [ 58  88  83]
  [ 48  78  73]
  ...
  [  8  17   7]
  [ 12  21  11]
  [ 11  20  10]]

 [[ 58  88  83]
  [ 62  92  87]
  [ 48  78  73]
  ...
  [ 10  18   8]
  [ 12  20  10]
  [  8  16   6]]

 ...

 [[ 96  98  99]
  [ 95  97  98]
  [ 95  97  98]
  ...
  [ 95 100  99]
  [ 90  95  94]
  [ 90  95  94]]

 [[ 96  98  99]
  [ 95  97  98]
  [ 95  97  98]
  ...
  [ 96 101 100]
  [ 92  97  96]
  [ 92  97  96]]

 [[ 96  98  99]
  [ 95  97  98]
  [ 95  97  98]
  ...
  [ 98 103 102]
  [ 94  99  98]
  [ 95 100  99]]]
